# Multi-agent Collaboration for Financial Analysis


In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [ ]:
import os
from utils import get_openai_api_key, get_serper_api_key

os.environ["OPENAI_API_KEY"] = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4.1-nano'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [ ]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-4.1-nano", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=False
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'NVIDIA',  # Example stock selection
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

# Agent: Data Analyst
## Task: Collect and analyze recent historical price data and market news for NVIDIA (ticker: NVDA). Use data from reliable sources such as Yahoo Finance, Nasdaq, and any available financial data APIs. Perform statistical modeling and apply machine learning techniques to identify trends and predict market movements. Provide insights and alerts about significant market opportunities or threats for NVIDIA, ensuring to include all relevant data and analysis results in the final output.


# Agent: Data Analyst
## Thought: Thought: To start this task, I need to gather recent historical price data and market news for NVIDIA (NVDA) from reliable sources like Yahoo Finance and Nasdaq. Then I can proceed with statistical modeling and machine learning analysis. First, I will collect recent historical price data for NVDA from Yahoo Finance.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://finance.yahoo.com/quote/NVDA/history?p=NVDA\"}"
## Tool

- Display the final result as Markdown.

In [18]:
from IPython.display import Markdown
Markdown(str(result))

Comprehensive Risk Analysis Report: Risks and Mitigation Strategies for NVIDIA Trading Strategies and Execution Plans

1. Introduction

This report evaluates the risks associated with the proposed trading strategies and execution plans for NVIDIA (NVDA) stock. The analysis covers potential market, operational, strategy-specific, and external risks inherent in the medium-risk day trading approaches outlined, namely Momentum-Volatility Swing Scalping, Pre-Event Volatility Breakout Trading, and Institutional Interest Fade and Reversal strategies. Mitigation recommendations are also provided to support effective risk management and enhance the probability of successful execution.

---

2. Overview of Proposed Strategies

- Momentum-Volatility Swing Scalping: Intraday trades based on short- to medium-term momentum shifts using RSI, MACD, and Bollinger Bands for entries and exits with tight stop losses.
- Pre-Event Volatility Breakout Trading: Trades ahead of key catalysts (earnings, product launches) based on breakouts or breakdowns confirmed by volume and ATR for position sizing.
- Institutional Interest Fade and Reversal: Counter-trend trades fading sharp moves driven by institutional block orders detected via Level 2 data and momentum indicators, with tight stops and quick exits.

---

3. Potential Risks

3.1 Market and Price Volatility Risks

- Volatility spikes around earnings or product launches may exceed ATR-based expectations, causing stop losses to be hit prematurely or slippage.
- Sudden macroeconomic or geopolitical events (e.g., regulatory changes, trade tensions) can cause unexpected price gaps and increased volatility.
- Rapid price movements driven by sector-wide news or competitor actions may invalidate momentum or reversal signals.
- Overbought/oversold indicators (RSI, MACD) may remain extreme longer than expected, leading to potential losses on fade trades.

3.2 Execution and Operational Risks

- Rapid intraday price movements require timely order execution; delays in order placement or data feed latency can cause missed opportunities or slippage.
- High frequency of trades in scalping strategy increases transaction costs, reducing profitability.
- Risk of misinterpretation of Level 2 data or volume spikes, leading to incorrect trade entries.
- Errors in setting stop losses or profit targets may lead to excessive losses or missed gains.

3.3 Strategy-Specific Risks

Momentum-Volatility Swing Scalping:
- False momentum signals during low liquidity periods or unusual volume spikes.
- Tight stops (0.5%-1%) can result in frequent small losses that accumulate.

Pre-Event Volatility Breakout Trading:
- False breakouts leading to stop loss triggers.
- Overleverage risk if position sizing is not adjusted properly to ATR or event volatility.
- Difficulty distinguishing genuine breakouts from noise.

Institutional Interest Fade and Reversal:
- Risk of fading strong fundamental moves justified by sector news, leading to losses.
- Limited window to execute fade trades; hesitation may reduce effectiveness.
- Difficulty in accurately detecting large block trades or institutional activity.

3.4 Portfolio and Risk Management Risks

- Exposure exceeding recommended 1%-2% per trade can increase drawdown risk.
- Holding positions overnight contradicts guidelines and increases risk from after-hours gaps.
- Inadequate monitoring of news or alerts may expose the portfolio to unanticipated risks.
- Overtrading can increase psychological and financial stress, impacting decision quality.

---

4. Mitigation Strategies

4.1 Market and Price Volatility Risk Mitigation

- Use dynamic position sizing with ATR and volatility measures to adjust trade sizes and stops based on current market conditions.
- Avoid trading during major macroeconomic announcements unrelated to NVIDIA to prevent unexpected volatility.
- Continuously monitor sector and competitor news to anticipate potential price impacts.
- Employ trailing stops to protect gains and limit losses amid volatile moves.

4.2 Execution and Operational Risk Mitigation

- Utilize reliable, low-latency trading platforms with real-time Level 2 data and order execution capabilities.
- Predefine and automate entry, exit, and stop loss orders where possible to minimize delays.
- Maintain strict discipline in following the defined stop loss and profit target rules.
- Regularly review trade performance metrics to identify and correct operational inefficiencies.

4.3 Strategy-Specific Risk Mitigation

Momentum-Volatility Swing Scalping:
- Avoid entering trades during unusual volume spikes unless momentum confirmation is strong.
- Limit number of scalping trades per day to manage transaction costs.
- Use multiple timeframe confirmations to reduce false signals.

Pre-Event Volatility Breakout Trading:
- Confirm breakouts with volume and momentum indicators before entry.
- Keep position sizes moderate and avoid overleveraging, especially near high-impact events.
- Prepare to exit quickly if breakout fails or price reverses.

Institutional Interest Fade and Reversal:
- Cross-verify institutional activity signals with sector fundamentals and news to avoid fading justified moves.
- Execute fade trades promptly upon signal confirmation to maximize effectiveness.
- Use very tight stop losses (within 0.5%) to limit drawdowns.

4.4 Portfolio and Risk Management Mitigation

- Adhere strictly to 1%-2% portfolio risk per trade limit.
- Avoid holding positions overnight; close all trades before market close.
- Stay updated with real-time news feeds and alerts for NVIDIA and the semiconductor sector.
- Maintain a trading journal to review decisions and refine strategies over time.
- Incorporate regular breaks to mitigate fatigue and maintain decision quality.

---

5. Conclusion

The proposed medium-risk trading strategies for NVIDIA stock leverage technical indicators, event-driven approaches, and institutional activity insights to capture intraday and event-related price movements. However, inherent market volatility, execution challenges, and strategy-specific pitfalls present notable risks.

Implementing the recommended mitigation strategies, including disciplined risk management, dynamic position sizing, strict stop loss adherence, and continuous market monitoring, will help minimize potential losses and enhance the effectiveness of the trading plans.

Adopting a cautious, well-informed approach aligned with the detailed execution plans and risk guidelines will optimize trading performance while safeguarding portfolio capital amidst NVIDIA’s dynamic market environment.

---

This concludes the comprehensive risk analysis report detailing potential risks and mitigation recommendations for trading NVIDIA stock based on the provided strategies and execution plans.